## Simulate Fish Passage

Average Fish Passage for Each Species

Ice Harbor: (.99 + .98 + .96 + .93)/4 = .965

Lower Monumental: (.99 + .99 + .95 + .93)/4 = .965

Little Goose: (.99 + .99 + .97 + .96)/4 = .9775

Lower Granite: TBD??

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
outflow_data = pd.read_csv('4200 Modified Average Daily Streamflows.csv')
outflow_data.rename(columns={'Ice Harbor Daily Streamflows (unit:cfs)': 'Ice Harbor outflow (cfs)'}, inplace=True)
outflow_data.rename(columns={'Lower Monumental (unit:cfs)': 'Lower Monumental outflow (cfs)'}, inplace=True)
outflow_data.rename(columns={'Little Goose (unit:cfs)': 'Little Goose outflow (cfs)'}, inplace=True)
outflow_data.rename(columns={'Lower Granite (unit:cfs)': 'Lower Granite outflow (cfs)'}, inplace=True)
outflow_data['date'] = pd.to_datetime(outflow_data['date'])

#ADD PRECIPITATION DATA
precip_data = pd.read_csv('NOAAprecipitation_data_LEWISTON_AIRPORT_ID.csv')
display(precip_data)

#will probably eventually need to cut our data s.t. it is all for the same length of time

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,SNOW,SNWD
0,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,1948-01-01,0.00,0.0,NaN
1,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,1948-01-02,0.00,0.0,0.0
2,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,1948-01-03,0.00,0.0,0.0
3,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,1948-01-04,0.06,0.0,0.0
4,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,1948-01-05,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...
28418,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,2025-10-21,0.00,0.0,0.0
28419,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,2025-10-22,0.00,0.0,0.0
28420,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,2025-10-23,0.00,0.0,0.0
28421,USW00024149,"LEWISTON AIRPORT, ID US",46.37458,-117.01529,435.8,2025-10-24,0.00,0.0,0.0


In [16]:
x=0
#added fish passage to res info
#added Watershed areas to res info
res_info = pd.DataFrame({'Names':['Ice Harbor','Lower Monumental','Little Goose','Lower Granite'],
                         'Reservoir surface area (m^2)':[9200*4047,6590*4047,10025*4047,8900*4047],
                         'Number of turbines':[6,6,6,6],
                         'Generation capacity (kW)':[603000,810000,903000,810000],
                         'Average tailwater elevation (m)':[x,x,x,x],
                         'Maximum pooling elevation (m)':[x,x,646.5,x],
                         'Fish Passage Rate (%)':[.965,.965,.9775,x],
                         'Watershed Area (acres)':[103_352,95_277,83_074,111_602]
                         })

# df = pd.DataFrame({'Ice Harbor':})
def simulate_fish_passage(df, keep):
    #df is dataframe containing Dam Name and inflow outflow data?
    if keep == 1:
        df['fish passage (%)'] = 1


## Simulate Runoff using Curve Number Method

$ Q = \frac{(P-I_a)^2}{(P-I_a+S)} $

$Q_v = QA$

From land use data:
CN = 84

$S = \frac{1000}{CN}-10 $ (convert inches to meters) = 0.0484 meter

$I_a = 0.2S $ = 0.00968 meter

In [ ]:
#Watershed Delineation Areas: HUC-12
#these were added to res_info above
Inflow_area = 111_602 #acres
Goose_WS_area = 83_074
Monumental_WS_area = 95_277  
Ice_WS_area = 103_352

In [ ]:
S = 0.0484 #meter
I = 0.00968 #meter
def calc_runoff(df,P):
    #takes in precip data as P and dam name dataframe as df
    #returns runoff volume
    Area = res_info[df]['Watershed Area (acres)'] #not sure if this is the right way to reference this
    if P <= I:
        Q = 0
    elif P > I:
        Q = (P-I)**2/(P-I+S)
    Q_v = Q * Area
    #do i need to add this to the existing streamflow data?
    df['Runoff volume (m^3)'] = Q

In [18]:
import reservoir_simulation

# simulate Little Goose reservoir
little_goose = pd.DataFrame(outflow_data['Little Goose outflow (cfs)'])
little_goose['inflow (cfs)'] = simulate_inflow(little_goose['Little Goose outflow (cfs)'])
little_goose.rename(columns={'Little Goose outflow (cfs)':'outflow (cfs)'}, inplace=True)
display(little_goose)
simulate_reservoir(little_goose,100000,0,'Little Goose')
display(little_goose)

NameError: name 'simulate_inflow' is not defined